# Tutorial evaluate taxonomy validation rules

This tutorial describes how to convert XBRL-instances to csv-, html- and pickle files per template.

We use Arelle, a open source package for processing XBRL. In addition this repository contains code to process the Solvency 2 and FTK instances efficiently.

In [1]:
from arelle import ModelManager, Cntlr, ModelXbrl, XbrlConst, RenderingEvaluator, \
                   ViewFileRenderedGrid, ModelFormulaObject
from arelle import PackageManager, FileSource

import src
import pandas as pd
import os
from os import listdir, walk, makedirs, environ
from os.path import isfile, join, exists, basename
from datetime import datetime

## Initialize the Arelle model manager

First we specify the directories with the taxonomy and instances. You can put your own instances in the data/instances directory, or you can specify here the directories that you want to use.

In [2]:
# the taxonomy should be data/taxonomy/arelle
# the instances you want to use should be in data/instances

XBRL_TAXONOMY_PATH = join('..', 'data', 'taxonomies')
XBRL_INSTANCES_PATH = join('..', 'data', 'instances')

LANGUAGE = "en-GB"
environ['XDG_CONFIG_HOME'] = XBRL_TAXONOMY_PATH

# The role defined in the model.xsd schema for resources representing codes of rows or columns is
euRCcode = 'http://www.eurofiling.info/xbrl/role/rc-code'

To process XBRL, we need a controller and a modelmanager object. 

In the controller you can specify logging. Here we have set logging to print in this notebook.

In [3]:
# Now we make a modelmanager
# logFileName = "logToPrint" -> logging is print to notebook
# logFileName = "arelle.log" -> logging is to filename (use .json or .xml for specific format)

controller = Cntlr.Cntlr(logFileName = "logToPrint")
controller.webCache.workOffline = True
controller.logger.messageCodeFilter = None

modelmanager = ModelManager.initialize(controller)
modelmanager.defaultLang = LANGUAGE
modelmanager.formulaOptions = ModelFormulaObject.FormulaOptions()
modelmanager.loadCustomTransforms()

## Initialize taxonomies

In [4]:
taxonomies = ['FTK Taxonomy 2.1.0_tcm46-386386.zip', 
              'EIOPA_SolvencyII_XBRL_Taxonomy_2.4.0_with_external_hotfix.zip']
PackageManager.init(controller)
for taxonomy in taxonomies:
    fs = FileSource.openFileSource(filename = join(XBRL_TAXONOMY_PATH, taxonomy))
    fs.open()
    PackageManager.addPackage(controller, fs.baseurl)
    fs.close()
PackageManager.rebuildRemappings(controller)
PackageManager.save(controller)

2021-03-05 16:11:53,428 [arelle.packageRewriteOverlap] Packages overlap the same rewrite start string http://www.eurofiling.info/ - EIOPA_SolvencyII_XBRL_Taxonomy_2.4.0_with_external_hotfix.zip , FTK Taxonomy 2.1.0_tcm46-386386.zip 

2021-03-05 16:11:53,430 [arelle.packageRewriteOverlap] Packages overlap the same rewrite start string http://www.xbrl.org/ - EIOPA_SolvencyII_XBRL_Taxonomy_2.4.0_with_external_hotfix.zip , FTK Taxonomy 2.1.0_tcm46-386386.zip 



## Read XBRL-instance in the modelmanager

Now we are able to read and process an XBRL-instance.

We read the example instances provided with the taxonomy.

In [5]:
# Solvency 2:
prefix = ""
# the example instance of the quarterly templates for solo
instance_name = 'qrs_240_instance.xbrl'  
# the example instance of the annual templates
# instance_name = 'aeb_240_instance.xbrl'

# FTK:
# prefix = "FTK."
# the example instance of the FTK assets templates
# instance_name = 'DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL'

## Validate instance from Arelle

It should be possible to validate the instance (performing the validation rules within the taxonomy) with Arelle with the following code. But we did not test this!

In [6]:
modelXbrl = modelmanager.load(join(XBRL_INSTANCES_PATH, instance_name))

In [7]:
controller = Cntlr.Cntlr(logFileName = "log.txt")
controller.webCache.workOffline = True
controller.setLogCodeFilter(None)
controller.logger.setLevel("ASSERTION-NOT-SATISFIED")
modelmanager.formulaOptions.traceVariableSetExpressionResult = True
# controller.startLogging()

In [8]:
modelmanager.validateInferDecimals = True
modelmanager.validateCalcLB = True

In [9]:
modelmanager.validate()

2021-03-05 16:12:53,630 [message:s2md_TV61-1] TV61-1: empty({T.99.01.01.01, c0080}) - qrs_240_instance.xbrl 11

2021-03-05 16:12:53,632 [message:s2md_TV37-2] TV37-2: if (not(empty({S.08.02.01.01, c0230}))) then not(empty({S.08.02.01.02, c0320})) - qrs_240_instance.xbrl 11

2021-03-05 16:13:31,059 [message:s2md_TV70-3] TV70-3: empty({S.01.02.01.01, r0992}) - qrs_240_instance.xbrl 11

2021-03-05 16:13:41,501 [message:s2md_TV15] TV15: si1559 doesn't follow "^LEI/[A-Z0-9]{20}$" or "^None" pattern -->Template 1: (All); Expression: si1559 like "^LEI/[A-Z0-9]{20}$" or "^None" - qrs_240_instance.xbrl 10

2021-03-05 16:13:41,509 [message:s2md_TV15] TV15: si1559 doesn't follow "^LEI/[A-Z0-9]{20}$" or "^None" pattern -->Template 1: (All); Expression: si1559 like "^LEI/[A-Z0-9]{20}$" or "^None" - qrs_240_instance.xbrl 10

2021-03-05 16:13:44,044 [message:s2md_TV34-2] TV34-2: if (not(empty({S.06.02.01.01, c0170}))) then not(empty({S.06.02.01.02, c0290})) - qrs_240_instance.xbrl 11

2021-03-05 16:14

KeyboardInterrupt: 